In [1]:
import geopandas as gpd
import pandas as pd
import json
from geojson import Feature, Point, FeatureCollection
cens = pd.read_excel('cens_final.xlsx')
geo = pd.read_excel('pobles.xlsx')

In [19]:
def normalitzar_columnes(df, columnes):
    for columnes in columnes:
        if columnes in df.columns:
            df[columnes] = df[columnes].str.strip()
            df[columnes] = df[columnes].str.upper()
    return df

#Columnes a normalitzar
columnes_cens = ['Municipi referencia', 'Indret desaparició', 'Lloc afusellament','Municipi localitzat']
columnes_geo = ['NOMBRE_ACTUAL']

#Funcio de normalització als dos dataframes
cens = normalitzar_columnes(cens, columnes_cens)
geo = normalitzar_columnes(geo, columnes_geo)

In [20]:
import numpy as np
#Transformació a str
cens['Municipi referencia'] = cens['Municipi referencia'].astype(str)
cens['Indret desaparició'] = cens['Indret desaparició'].astype(str)
cens['Lloc afusellament'] = cens['Lloc afusellament'].astype(str)
cens['Municipi localitzat'] = cens['Municipi localitzat'].astype(str)
#Eliminació duplicats
coo_pob_naixament = pd.DataFrame(np.unique(cens['Municipi referencia']), columns=['Municipi'])
coo_pob_desaparicio = pd.DataFrame(np.unique(cens['Indret desaparició']), columns=['Municipi'])
coo_pod_afusellament = pd.DataFrame(np.unique(cens['Lloc afusellament']), columns=['Municipi'])
coo_pob_localizat = pd.DataFrame(np.unique(cens['Municipi localitzat']), columns=['Municipi'])
#Concatenació municipis i indrets
coo_concatenat_munidicpis = pd.concat([coo_pob_naixament, coo_pob_localizat])
coo_concatenat_indrets = pd.concat([coo_pob_desaparicio, coo_pod_afusellament])
#Eliminació duplicats
totalmunicipis = coo_concatenat_munidicpis.drop_duplicates().reset_index(drop=True)
totalindrets = coo_concatenat_indrets.drop_duplicates().reset_index(drop=True)

In [21]:
#Unio municipi i indrets
unio = totalindrets.merge(totalmunicipis, on='Municipi', how='outer', indicator=True)
#Filtrem valors per obtenir només indrets que no estan a municipis
indrets = unio[unio['_merge'] == 'left_only']
# Quitar la columna del indicador
indrets = indrets.drop(columns=['_merge'])

In [22]:
from fuzzywuzzy import process
#Funció per buscar coordenades a l'arxiu cartografic 
def buscar_coordenades(nom, df_municipis):
    #Busca coicidencia exacta
    resultat_exacte = df_municipis[df_municipis['NOMBRE_ACTUAL'].str.lower() == nom.lower()]
    if not resultat_exacte.empty:
        return resultat_exacte.iloc[0]['LONGITUD_ETRS89'], resultat_exacte.iloc[0]['LATITUD_ETRS89']
    
    # Si no es troba coicidencia exacte es fa busqueda difusa
    coincidencia = process.extractOne(nom, df_municipis['NOMBRE_ACTUAL'], score_cutoff=80)
    if coincidencia:
        coordenades = df_municipis.loc[df_municipis['NOMBRE_ACTUAL'] == coincidencia[0], ['LONGITUD_ETRS89', 'LATITUD_ETRS89']]
        return coordenades.iloc[0]['LONGITUD_ETRS89'], coordenades.iloc[0]['LATITUD_ETRS89']
    return None, None

In [23]:
# Aplicar la función a la columna 'Municipi naixement' del DataFrame 'coordenadas_naixament_df'
totalmunicipis['Coordenades'] = totalmunicipis['Municipi'].apply(lambda x: buscar_coordenades(x, geo))
totalmunicipis.head()

,Municipi,Coordenades
0,A GUARDA,"(-8.874218844, 41.90117013)"
1,ABADIA,"(-5.97785806, 40.25953202)"
2,ABARAN,"(-1.40118044, 38.20259944)"
3,ABELLA DE LA CONCA,"(1.09177173, 42.16142117)"
4,ABENFIGO,"(-0.20921861, 40.27646952)"


In [24]:
totalmunicipis.to_excel('totalmunicipis.xlsx')

In [25]:
indrets.to_excel('totalindrets.xlsx')

In [26]:
totalmunicipis[['Longitud', 'Latitud']] = pd.DataFrame(totalmunicipis['Coordenades'].tolist(), index=totalmunicipis.index)
totalmunicipis.drop('Coordenades', axis=1, inplace=True)
totalmunicipis.rename(columns={'Municipi': 'Nombre'}, inplace=True)

In [27]:
#Unio dataframes.
indrets_GOOGLE = pd.read_excel('GOOGLE_indrets.xlsx')
total_ubicacions = pd.concat([totalmunicipis, indrets_GOOGLE], ignore_index=True)

In [28]:
#Creació llista GeoJSON
features = []
for _, row in total_ubicacions.iterrows():
    #comprovació de nuls
    if pd.notnull(row['Latitud']) and pd.notnull(row['Longitud']):
        point = Point((row['Longitud'], row['Latitud']))
        feature = Feature(geometry=point, properties={"Nombre": row['Nombre']})
        features.append(feature)

#string de GeoJSON
feature_collection = FeatureCollection(features)
geojson_str = json.dumps(feature_collection)

#Guardar Geojson
geojson_filename = 'total_ubicacions.geojson'  
with open(geojson_filename, 'w') as f:
    f.write(geojson_str)

In [29]:
#Lectura arxiu cens
cens1 = pd.read_excel('cens_final.xlsx')

In [30]:
#Crear DATAFRAME AMB CONFIGURACIÓ PER CREAR LINEA UNIO ENTRE MUNICIPIS A TABLEU
df1_melted = cens1.melt(id_vars='Id Afectat', 
                        value_vars=['Municipi referencia', 'Indret desaparició', 'Lloc afusellament', 'Municipi localitzat'],
                        var_name='Ordre', 
                        value_name='Municipi')

#Ordre per categoria de municipi
orden_map = {
    'Municipi referencia': 1,
    'Indret desaparició': 2,
    'Lloc afusellament': 3,
    'Municipi localitzat': 4
}

df1_melted['Ordre'] = df1_melted['Ordre'].map(orden_map)

#Eliminació Nulls
df1_melted.dropna(subset=['Municipi'], inplace=True)

# Ordenació 
df1_melted.sort_values(by=['Id Afectat', 'Ordre'], inplace=True)

#Restablir index
df1_melted.reset_index(drop=True, inplace=True)

In [31]:
df1_melted

,Id Afectat,Ordre,Municipi
0,1,1,PUIGCERDÀ
1,1,2,FRONT DE L'EBRE
2,2,1,BARCELONA
3,2,2,FRONT D'ARAGÓ
4,3,1,CUEVAS DE VERA
...,...,...,...
10582,5307,2,VALLS
10583,5308,1,ÑORA
10584,5309,1,SANAHUJA
10585,5310,1,PIERA


In [32]:
import pandas as pd
import json

# Cargar arxiu Geojson amb totes les ubicacions
with open('total_ubicacions.geojson', 'r', encoding='utf-8') as f:
    geojson_data = json.load(f)

# Convertir GeoJSON a DataFrame
coordenades_list = [
    {
        'Municipi': feature['properties']['Nombre'],
        'Latitud': feature['geometry']['coordinates'][1],
        'Longitud': feature['geometry']['coordinates'][0]
    }
    for feature in geojson_data['features']
]

#Dataframe de coordenades
df_coordenadas = pd.DataFrame(coordenades_list)

In [33]:
# Unió df orde per Tableu amb df coordenades
df1_melted['Municipi'] = df1_melted['Municipi'].str.strip()
df_coordenadas['Municipi'] = df_coordenadas['Municipi'].str.strip()
df_rutes = pd.merge(df1_melted, df_coordenadas, on='Municipi', how='left')
df_rutes

,Id Afectat,Ordre,Municipi,Latitud,Longitud
0,1,1,PUIGCERDÀ,42.432641,1.926340
1,1,2,FRONT DE L'EBRE,41.848662,-1.657646
2,2,1,BARCELONA,41.384247,2.176349
3,2,2,FRONT D'ARAGÓ,NaN,NaN
4,3,1,CUEVAS DE VERA,37.246753,-1.868775
...,...,...,...,...,...
10584,5307,2,VALLS,41.283889,1.247384
10585,5308,1,ÑORA,42.569231,-2.470085
10586,5309,1,SANAHUJA,39.900169,-0.340497
10587,5310,1,PIERA,41.518061,1.743458


In [34]:
import pandas as pd
import json
# Funció per crear GEojson del dataframe df_rutes
def crear_geojson(df):
    # Crear la lista de Features
    features = []
    for _, row in df.iterrows():
        feature = {
            "type": "Feature",
            "properties": {
                "Id Afectat": row['Id Afectat'],
                "Ordre": row['Ordre'],
                "Municipi": row['Municipi']
            },
            "geometry": {
                "type": "Point",
                "coordinates": [row['Longitud'], row['Latitud']]
            }
        }
        features.append(feature)

    # Crear el FeatureCollection
    feature_collection = {
        "type": "FeatureCollection",
        "features": features
    }

    return feature_collection

#Creem Geojson de rutes 
geojson_data = crear_geojson(df_rutes)

# Guardar arxiu geojson
geojson_filename = 'ruta_final.geojson'
with open(geojson_filename, 'w', encoding='utf-8') as f:
    json.dump(geojson_data, f, ensure_ascii=False)